In [1]:
import pandas as pd
import numpy as np

# 1.0 Cleaning Data
## 1.1 Splitting violation codes out
First I want to break out the violations into their own table so we can extract the violation codes

In [2]:
healthdf = pd.read_csv('/home/michael/Downloads/Food_Inspections.csv')

In [3]:
healthdf.head()

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location
0,2144504,TAQUERIA EL RANCHITO,TAQUERIA EL RANCHITO,60184.0,Restaurant,Risk 1 (High),2829 N MILWAUKEE AVE,CHICAGO,IL,60618.0,02/02/2018,Canvass Re-Inspection,Pass,31. CLEAN MULTI-USE UTENSILS AND SINGLE SERVIC...,41.932657,-87.713056,"(41.93265693660558, -87.71305610116421)"
1,2144489,EL TACONAZO JR.,EL TACONAZO JR.,2354772.0,Restaurant,Risk 1 (High),4256 W FULLERTON AVE,CHICAGO,IL,60639.0,02/02/2018,Canvass,Pass w/ Conditions,3. POTENTIALLY HAZARDOUS FOOD MEETS TEMPERATUR...,41.924508,-87.733980,"(41.92450769084945, -87.73398032565292)"
2,2144480,BRICK HOUSE BAR AND GRILL,BRICK HOUSE BAR AND GRILL,1969907.0,Restaurant,Risk 1 (High),3152-3154 W IRVING PARK RD,CHICAGO,IL,60618.0,02/02/2018,Canvass Re-Inspection,Pass,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,41.954032,-87.707720,"(41.95403155441393, -87.70771986355506)"
3,2144477,THE VEGGIE GRILL,THE VEGGIE GRILL,2563666.0,NaN,Risk 1 (High),614 W DIVERSEY AVE,CHICAGO,IL,60614.0,02/02/2018,License,Not Ready,NaN,41.932917,-87.644572,"(41.93291687750861, -87.64457163885304)"
4,2144474,SMOQUE BBQ,SMOQUE BBQ,1694866.0,Restaurant,Risk 1 (High),3800 N PULASKI RD,CHICAGO,IL,60641.0,02/02/2018,Canvass,Pass w/ Conditions,38. VENTILATION: ROOMS AND EQUIPMENT VENTED AS...,41.950073,-87.727657,"(41.95007328750793, -87.72765652046023)"


In [4]:
healthdf.iloc[0]['Violations']

'31. CLEAN MULTI-USE UTENSILS AND SINGLE SERVICE ARTICLES PROPERLY STORED: NO REUSE OF SINGLE SERVICE ARTICLES - Comments: OBSERVED CUPS USED TO DISPENSE FOOD IN THE CONTAINERS OF CHEESE AND GIARDINNARA IN THE COLD HOLDING REFRIGERATION UNIT. INSTRUCTED TO USE HANDLED SCOOPS TO DISPENSE FOOD AND TO STORE HANDLED SCOOPS EITHER IN THE FOOD WITH THE HANDLE FACING UP OUT OF THE FOOD OR IN A SEPARATE CONTAINER IN BETWEEN USE. | 35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTRUCTED PER CODE: GOOD REPAIR, SURFACES CLEAN AND DUST-LESS CLEANING METHODS - Comments: MUST DETAIL CLEAN THE FOOD SPLATTER ON THE WALLS BEHIND THE HOT HOLDING UNITS AND THE REFRIGERATION UNITS ACROSS THE COOKING EQUIPMENT ON THE LINE. \n | 36. LIGHTING: REQUIRED MINIMUM FOOT-CANDLES OF LIGHT PROVIDED, FIXTURES SHIELDED - Comments: OBSERVED MISSING LIGHT SHIELDS ABOVE THE PREP AREA IN THE KITCHEN. INSTRUCTED TO PROVIDE MISSING SHIELDS TO PROTECT LIGHT FIXTURES AT ALL TIMES.'

In [5]:
violations = healthdf[['Inspection ID','Violations']]
violations = violations.dropna()
violations.shape

(131495, 2)

In [6]:
def split_codes(df):
    '''
    function to split the violations into an inspection id, violation code, 
    violation code description and the comment on the violation
    '''
    result = pd.DataFrame()
    for index, rows in df.iterrows():
        x = pd.DataFrame(rows['Violations'].split('|'),columns = ['fulldesc'])
        x['fulldesc'] = x['fulldesc'].str.strip()
        x['Inspection ID'] = rows['Inspection ID']
        x['vcode'] = x['fulldesc'].str.split('.').str[0]
        x['codedesc'] = x['fulldesc'].str.split(' - ').str[0]
        x['comment'] = x['fulldesc'].str.split(' - ').str[1]
        x['comment'] = x['comment'].str[9:]
        result = result.append(x)
    return result[['Inspection ID','vcode','codedesc','comment']]

In [ ]:
viodf = split_codes(violations)

In [ ]:
viodf.tail()

In [ ]:
viodf.to_pickle('/home/michael/chicagohealthinspections/data/violations.pkl')

## 1.2 Clean Business Info
Now we need to fix some of the business info before we add it all to the database

In [ ]:
businfodf = pd.read_pickle('/home/michael/chicagohealthinspections/data/businessinfo.pkl')

In [ ]:
businfodf.head()